In [ ]:
# Objective

To have a machine-readable set of data on the State Records Office's collection of scanned maps and survey plans, with
each object having a full set of metadata available, including a rough spatial location to enable end-users to
choose objects of local interest to georectify.

In [ ]:
# For want of an API, let's scrape the digital objects collection from SRO.

https://github.com/geogeeks-au/maps-for-lost-towns/issues/8

In [ ]:
from bs4 import BeautifulSoup
import requests
import time
from pprintpp import pprint as pp

page = 225
scraping = True
while scraping is True:
    # @TODO Determine what a nice wait period is to stop SRO's server from falling over
    time.sleep(1)
    
    r = requests.get(
        "https://archive.sro.wa.gov.au/index.php/digitalobject/browse?page=%s&limit=30&sort=identifier" % (page)
    )
    soup = BeautifulSoup(r.text, "lxml")
    anchors = soup.select("div.preview > a")
        
    if len(anchors) == 0:
        scraping = False
    else:
        print "Page #%s, #%s Items" % (page, len(anchors))

        print anchors[0]["href"]
        r_item = requests.get("https://archive.sro.wa.gov.au%s" % (anchors[0]["href"]))
        item_soup = BeautifulSoup(r_item.text, "lxml")
        fields = item_soup.select("div#content > section > div.field")
        print len(fields)
        for field in fields:
            print "%s: %s" % (field.find("h3").text.strip(), field.find("div").text.strip())
            print
        
        page += 1
    
    if page >= 230:
        print "die"
        break

In [ ]:
# Loot our local cache of images and extract all of the meaningful information we can about them

https://github.com/geogeeks-au/maps-for-lost-towns/issues/10
    
All done! See https://github.com/geogeeks-au/maps-for-lost-towns/blob/master/scrapers/

In [ ]:
# Now let's stitch the two together so we can see if the files we have can be associated with SRO's digital objects collection

https://github.com/geogeeks-au/maps-for-lost-towns/issues/11

In [ ]:
# OK, now finally let's try to assign a spatial location to these

https://github.com/geogeeks-au/maps-for-lost-towns/issues/9